# Exploratory Data Analysis with Spark

In this notebook, we will be using Spark to perform some data analysis on our disaster messages dataset. We'll be using our training dataset, previously processed in 2 separate notebooks, namely 'Data Augmentation.ipynb' and 'Data Cleaning.ipynb'.

### Setting-up the Environment

First, we will import the required libraries and set-up a Spark session.

In [1]:
# Import libraries and find spark
import findspark
findspark.init('spark/')

import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.functions import col,isnan, when, count, isnull, udf, struct, col
from textblob import TextBlob
from pyspark.sql.types import * 

In [2]:
# Start Spark session
spark = SparkSession.builder.appName('EDA').getOrCreate()
spark

### Exploring the Data

Next, we will start by using Spark to get an overview of the data, such as its schema, number of null values, as well as some statistical measures of the features.

As an added note, the 'train.csv' file's delimiter was changed from the default "," to "|". This is to prevent any errors while reading the data, as our disaster messages have many commas, and in some cases, these could be misinterpreted as column separators. The codes to change the delimiters are not provided here.

In [3]:
# Read data
df = spark.read.option('header','true').option('delimiter','|').csv('TrainSpark.csv',inferSchema=True)

We will print out the dataset schema below.

In [4]:
# Print Schema
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ID: integer (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- labeled: integer (nullable = true)
 |-- message: string (nullable = true)
 |-- original: string (nullable = true)
 |-- language: string (nullable = true)
 |-- related: integer (nullable = true)
 |-- request: integer (nullable = true)
 |-- aid_related: integer (nullable = true)
 |-- medical_help: integer (nullable = true)
 |-- medical_products: integer (nullable = true)
 |-- search_and_rescue: integer (nullable = true)
 |-- security: integer (nullable = true)
 |-- military: integer (nullable = true)
 |-- water: integer (nullable = true)
 |-- food: integer (nullable = true)
 |-- shelter: integer (nullable = true)
 |-- clothing: integer (nullable = true)
 |-- money: integer (nullable = true)
 |-- missing_people: integer (nullable = true)
 |-- refugees: integer (nullable = true)
 |-- death: integer (nullable = true)
 |-- other_aid: integer (nullable = true)
 |-- infrastru

Next, we will extract a sample row, then the first 5 columns, to get a sense of the dataset. We also remove the unnecessary column.

In [5]:
# Extract sample row
df.show(1, vertical = True)

-RECORD 0--------------------------------------
 _c0                    | 0                    
 ID                     | 1                    
 date                   | 2010-01-01 00:00:00  
 labeled                | 0                    
 message                | With the cooperat... 
 original               | null                 
 language               | en                   
 related                | 1                    
 request                | 0                    
 aid_related            | 1                    
 medical_help           | 0                    
 medical_products       | 0                    
 search_and_rescue      | 0                    
 security               | 0                    
 military               | 0                    
 water                  | 0                    
 food                   | 0                    
 shelter                | 0                    
 clothing               | 0                    
 money                  | 1             

In [6]:
# View first 3 columns
df.select(['date','labeled','message','original','language']).show()

+-------------------+-------+--------------------+--------------------+--------+
|               date|labeled|             message|            original|language|
+-------------------+-------+--------------------+--------------------+--------+
|2010-01-01 00:00:00|      0|With the cooperat...|                null|      en|
|2010-01-01 00:00:00|      1|PEWODEN FIFTH SEC...|Pewoden 5em Seksy...|      ht|
|2010-01-01 00:00:00|      1|Today on a call w...|                null|      en|
|2010-01-01 00:00:00|      0|YANGON, Jul 08, 2...|                null|      en|
|2010-01-01 00:00:00|      1|Throughout the ye...|                null|      en|
|2010-01-01 00:00:00|      0|We have no pot in...|Nou pa gen chabon...|      ht|
|2010-01-01 00:00:00|      1|**May 12** A 7.3 ...|                null|      en|
|2010-01-02 00:00:00|      0|We would like to ...|renmen pou yo ta ...|      ht|
|2010-01-02 00:00:00|      0|By secret ballot,...|                null|      en|
|2010-01-02 00:00:00|      0

In [7]:
# Drop unnecessary column
df = df.drop('_c0')

We print out an overview of the dataset below.

In [8]:
# Describe
df.describe().toPandas()

,summary,ID,labeled,message,original,language,related,request,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,count,20878,20878,20878,8074,20878,20878,20878,20878,20878,...,20878,20878,20878,20878,20878,20878,20878,20878,20878,20878
1,mean,10528.289778714436,0.2986876137561069,None,None,None,0.7565379825653799,0.17051441708975956,0.4159881214675735,0.0792221477152984,...,0.012357505508190439,0.047322540473225407,0.28091771242456176,0.08128173196666347,0.09450138902193697,0.011447456652936104,0.09478877287096465,0.020356355972794327,0.05450713669891752,0.19202030845866463
2,stddev,6073.987365748489,0.45769351845278344,None,None,None,0.42918187991325973,0.37609310753877667,0.4929032776223667,0.27009163835821753,...,0.11047797142759565,0.21233293927178754,0.44945814836508385,0.2732738350419775,0.2925320073185589,0.10638117521803214,0.2929299769218409,0.14121943901943926,0.22702109440957874,0.39389838937222443
3,min,1,0,,a,af,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,max,21046,1,| News Update | Serious loss of life expected ...,zwen medikal nouriti dlo avan mwen te abite ma...,zu,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


We observe that there is one or more occurence(s) where the message column is blank. This does not provide additional value for us, and we will remove it later on. But for now, we'll leave it as it is.

Next, we'll see how many null values are present in our data.

In [9]:
# Find the number of null values
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns[3:]]).toPandas()

,message,original,language,related,request,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0,12804,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Below, we will create a new column to show the total number of categories, each message is classified into. This column is named 'Sum'.

In [10]:
# Add new column to show number of categories for each message
col_list = df.columns[7:]
df = df.withColumn('Sum',sum([col(c) for c in col_list]))
df.limit(5).toPandas()

,ID,date,labeled,message,original,language,related,request,aid_related,medical_help,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,Sum
0,1,2010-01-01,0,"With the cooperation of First Hawaiian Bank, t...",None,en,1,0,1,0,...,0,1,0,1,0,0,0,1,0,5
1,2,2010-01-01,1,PEWODEN FIFTH SECTION OF THE DEPARTEMEN OF L'A...,Pewoden 5em Seksyon Depatman Atibonit ap fe no...,ht,1,0,0,0,...,0,1,0,0,0,1,0,0,1,3
2,3,2010-01-01,1,"Today on a call with Dr. Chan, Director Genera...",None,en,1,0,1,1,...,0,0,0,0,0,0,0,0,0,3
3,4,2010-01-01,0,"YANGON, Jul 08, 2008 (Xinhua via COMTEX News N...",None,en,1,0,1,0,...,1,1,1,1,0,0,0,0,0,11
4,5,2010-01-01,1,Throughout the year there were growing signs o...,None,en,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2


### Queries Using Spark SQL

In this section, we will use Spark SQL to perform some data analysis on our dataset. We will see if we can obtain any insights on the difference in message characteristics, based on their original language, relevancy, etc.

First, we create a temporary view to query on, then try out a simple query by selecting specific columns from the dataset.

In [11]:
# Create temporary view
df.createOrReplaceTempView('MSGTable')

In [12]:
# Test query
query = "SELECT date, message, language FROM MSGTable"
spark.sql(query).show()

+-------------------+--------------------+--------+
|               date|             message|language|
+-------------------+--------------------+--------+
|2010-01-01 00:00:00|With the cooperat...|      en|
|2010-01-01 00:00:00|PEWODEN FIFTH SEC...|      ht|
|2010-01-01 00:00:00|Today on a call w...|      en|
|2010-01-01 00:00:00|YANGON, Jul 08, 2...|      en|
|2010-01-01 00:00:00|Throughout the ye...|      en|
|2010-01-01 00:00:00|We have no pot in...|      ht|
|2010-01-01 00:00:00|**May 12** A 7.3 ...|      en|
|2010-01-02 00:00:00|We would like to ...|      ht|
|2010-01-02 00:00:00|By secret ballot,...|      en|
|2010-01-02 00:00:00|By August 29, kha...|      en|
|2010-01-02 00:00:00|I am in Carefour ...|      ht|
|2010-01-02 00:00:00|As of Monday, Feb...|      en|
|2010-01-02 00:00:00|Here they live in...|      en|
|2010-01-02 00:00:00|Slt. M sends cond...|      ht|
|2010-01-03 00:00:00|i'd like to find ...|      ht|
|2010-01-03 00:00:00|Tens of thousands...|      en|
|2010-01-03 

Next, we extract the total messages by dates and month, then sort them by descending order.

In [13]:
# Messages by date
query = "SELECT date, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY date ORDER BY TotalMessages DESC"
spark.sql(query).show()

+-------------------+-------------+
|               date|TotalMessages|
+-------------------+-------------+
|2018-08-28 00:00:00|           19|
|2014-08-01 00:00:00|           17|
|2014-05-20 00:00:00|           15|
|2011-03-11 00:00:00|           15|
|2011-11-22 00:00:00|           14|
|2017-08-29 00:00:00|           14|
|2015-12-17 00:00:00|           14|
|2010-12-18 00:00:00|           14|
|2010-05-10 00:00:00|           14|
|2019-09-21 00:00:00|           14|
|2018-08-15 00:00:00|           14|
|2012-06-02 00:00:00|           14|
|2017-02-01 00:00:00|           14|
|2014-11-25 00:00:00|           14|
|2011-01-08 00:00:00|           13|
|2014-10-31 00:00:00|           13|
|2015-05-01 00:00:00|           13|
|2010-04-22 00:00:00|           13|
|2012-11-17 00:00:00|           13|
|2011-01-02 00:00:00|           13|
+-------------------+-------------+
only showing top 20 rows



In [14]:
# Messages by month and year
query = "SELECT MONTH(date) AS Mth, YEAR(date) AS Yr, COUNT(ID) AS TotalMessages FROM MSGTable GROUP BY Mth, Yr ORDER BY TotalMessages DESC"
spark.sql(query).show()

+---+----+-------------+
|Mth|  Yr|TotalMessages|
+---+----+-------------+
|  6|2019|          205|
|  8|2018|          204|
|  5|2011|          200|
| 11|2012|          200|
|  4|2010|          200|
|  7|2015|          199|
|  3|2010|          198|
| 10|2016|          196|
|  3|2016|          195|
|  6|2017|          194|
|  1|2012|          194|
| 12|2015|          194|
|  1|2011|          193|
|  9|2019|          193|
|  9|2015|          193|
|  8|2010|          192|
|  5|2019|          191|
| 11|2011|          191|
|  1|2016|          190|
|  6|2012|          189|
+---+----+-------------+
only showing top 20 rows



We also group the messages by languages to see the most popular languages used.

In [15]:
# Messages by language
query = "SELECT language, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY language ORDER BY TotalMessages DESC"
spark.sql(query).show()

+--------+-------------+
|language|TotalMessages|
+--------+-------------+
|      en|        13178|
|      ht|         5084|
|      fr|         1464|
|      nl|          144|
|      id|          109|
|      ms|           94|
|      eo|           83|
|      es|           73|
|      fi|           52|
|      br|           51|
|      sl|           47|
|      it|           45|
|      rw|           44|
|      sw|           42|
|      no|           38|
|      tl|           36|
|      jv|           28|
|      de|           26|
|      da|           24|
|      eu|           23|
+--------+-------------+
only showing top 20 rows



We can see that English is the most popular language by a large margin, followed by Haiti, then French. Next, we find out the number of labeled and unlabelled messages for each language.

In [16]:
# Messages by language and labeled category
query = "SELECT language, labeled, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY language,labeled ORDER BY TotalMessages DESC"
spark.sql(query).show()

+--------+-------+-------------+
|language|labeled|TotalMessages|
+--------+-------+-------------+
|      en|      0|         9236|
|      en|      1|         3942|
|      ht|      0|         3527|
|      ht|      1|         1557|
|      fr|      0|         1080|
|      fr|      1|          384|
|      nl|      0|          100|
|      id|      0|           73|
|      ms|      0|           64|
|      eo|      0|           59|
|      es|      0|           48|
|      nl|      1|           44|
|      fi|      0|           37|
|      id|      1|           36|
|      br|      0|           36|
|      sl|      0|           35|
|      rw|      0|           32|
|      it|      0|           31|
|      ms|      1|           30|
|      sw|      0|           27|
+--------+-------+-------------+
only showing top 20 rows



Following on, we can use the query below to observe the percentage of labeled messages for each language. We can see that this figure is around 70%.

In [17]:
# Messages by language and labeled category, along with the percentage
query = '''
WITH LangTotal As
(
    SELECT language, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY language ORDER BY TotalMessages DESC
),
LangLabeled As
(
    SELECT language, labeled, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY language,labeled ORDER BY TotalMessages DESC
)
SELECT
    L.language, L.labeled, L.TotalMessages, L.TotalMessages / T.TotalMessages * 100.0 As Percentage
FROM
    LangLabeled L
LEFT JOIN
    LangTotal T
ON
    L.language = T.language
ORDER BY T.TotalMessages DESC, L.labeled
'''

spark.sql(query).show()

+--------+-------+-------------+------------------+
|language|labeled|TotalMessages|        Percentage|
+--------+-------+-------------+------------------+
|      en|      0|         9236| 70.08650781605706|
|      en|      1|         3942|29.913492183942935|
|      ht|      0|         3527| 69.37450826121164|
|      ht|      1|         1557| 30.62549173878836|
|      fr|      0|         1080| 73.77049180327869|
|      fr|      1|          384|26.229508196721312|
|      nl|      0|          100| 69.44444444444444|
|      nl|      1|           44|30.555555555555557|
|      id|      0|           73| 66.97247706422019|
|      id|      1|           36| 33.02752293577982|
|      ms|      0|           64| 68.08510638297872|
|      ms|      1|           30|31.914893617021278|
|      eo|      0|           59| 71.08433734939759|
|      eo|      1|           24|28.915662650602407|
|      es|      0|           48| 65.75342465753424|
|      es|      1|           25| 34.24657534246575|
|      fi|  

Next, we extract the number and percentage of related messages for each language. A related message is when the message is a relevant emergency ticket.

In [18]:
# Messages by language and related category, along with the percentage
query = '''
WITH LangTotal As
(
    SELECT language, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY language ORDER BY TotalMessages DESC
),
RelLabeled As
(
    SELECT language, related, COUNT(ID) as TotalMessages FROM MSGTable GROUP BY language, related ORDER BY TotalMessages DESC
)
SELECT
    L.language, L.related, L.TotalMessages, L.TotalMessages / T.TotalMessages * 100.0 As Percentage
FROM
    RelLabeled L
LEFT JOIN
    LangTotal T
ON
    L.language = T.language
ORDER BY T.TotalMessages DESC, L.related DESC
'''

spark.sql(query).show()

+--------+-------+-------------+------------------+
|language|related|TotalMessages|        Percentage|
+--------+-------+-------------+------------------+
|      en|      1|        10752| 81.59052967066323|
|      en|      0|         2426|18.409470329336774|
|      ht|      1|         3574| 70.29897718332022|
|      ht|      0|         1510|29.701022816679778|
|      fr|      1|          764| 52.18579234972678|
|      fr|      0|          700| 47.81420765027322|
|      nl|      1|           87|60.416666666666664|
|      nl|      0|           57| 39.58333333333333|
|      id|      1|           71| 65.13761467889908|
|      id|      0|           38|34.862385321100916|
|      ms|      1|           72| 76.59574468085107|
|      ms|      0|           22|23.404255319148938|
|      eo|      1|           43| 51.80722891566265|
|      eo|      0|           40| 48.19277108433735|
|      es|      1|           43|  58.9041095890411|
|      es|      0|           30|  41.0958904109589|
|      fi|  

From the 20 most popular languages, we observe that English has the highest related percentage, whereas Finnish has the lowest. Finally, we find out the average number of classes each message is classified into, for each language. 

In [19]:
# Messages by language, along with average number of classes for each message
query = "SELECT language, COUNT(ID) AS Total_Messages, AVG(MSGTable.Sum) AS MessageClassAvg FROM MSGTable GROUP BY language ORDER BY Total_Messages DESC"
spark.sql(query).show()

+--------+--------------+------------------+
|language|Total_Messages|   MessageClassAvg|
+--------+--------------+------------------+
|      en|         13178| 2.555091819699499|
|      ht|          5084|2.4878048780487805|
|      fr|          1464|1.4624316939890711|
|      nl|           144|1.9791666666666667|
|      id|           109| 2.091743119266055|
|      ms|            94| 3.617021276595745|
|      eo|            83|1.2048192771084338|
|      es|            73| 1.356164383561644|
|      fi|            52|1.3461538461538463|
|      br|            51|1.3725490196078431|
|      sl|            47|2.6382978723404253|
|      it|            45|1.1333333333333333|
|      rw|            44| 4.454545454545454|
|      sw|            42|3.4285714285714284|
|      no|            38| 1.131578947368421|
|      tl|            36|2.1666666666666665|
|      jv|            28|              3.75|
|      de|            26|1.2692307692307692|
|      da|            24|0.6666666666666666|
|      eu|

From the 20 most popular languages, we observe that Javanese is highest in this category, whereas Dannish is lowest.

### Text Data Analysis with Spark

In this final section, we will be making use of Spark's UDF to apply functions defined using other Python libraries, to our Spark dataframe. We'll be focusing more on analysing the text-related features here.

First, we create a duplicate dataframe, with only relevant features, to perform our operations on.

In [20]:
# Select specific columns as new df
df_anl = df.select(['date', 'labeled', 'message', 'language', 'related'])
df_anl.show()

+-------------------+-------+--------------------+--------+-------+
|               date|labeled|             message|language|related|
+-------------------+-------+--------------------+--------+-------+
|2010-01-01 00:00:00|      0|With the cooperat...|      en|      1|
|2010-01-01 00:00:00|      1|PEWODEN FIFTH SEC...|      ht|      1|
|2010-01-01 00:00:00|      1|Today on a call w...|      en|      1|
|2010-01-01 00:00:00|      0|YANGON, Jul 08, 2...|      en|      1|
|2010-01-01 00:00:00|      1|Throughout the ye...|      en|      1|
|2010-01-01 00:00:00|      0|We have no pot in...|      ht|      1|
|2010-01-01 00:00:00|      1|**May 12** A 7.3 ...|      en|      1|
|2010-01-02 00:00:00|      0|We would like to ...|      ht|      1|
|2010-01-02 00:00:00|      0|By secret ballot,...|      en|      0|
|2010-01-02 00:00:00|      0|By August 29, kha...|      en|      0|
|2010-01-02 00:00:00|      0|I am in Carefour ...|      ht|      1|
|2010-01-02 00:00:00|      1|As of Monday, Feb..

We define 4 helper functions below to help us with our analysis. The first and second helper functions calculate the number of words and characters in each message, respectively, whereas the third helper function calculates the word density, which is obtained by dividing the number of words by the number of characters. Finally, in the fourth helper function, we determine the polarity of each message by using the textblob library in Python.

In [21]:
# Define functions in Python

# Get number of words
def word_count(msg):
    word_count = len(msg.split())
    
    return word_count

# Get number of chars
def char_count(msg):
    char_count = len(msg.replace(" ",""))
    
    return char_count

# Divide word count by char count to obtain word density
def word_density(msg):
    word_count = len(msg.split())
    char_count = len(msg.replace(" ",""))
    word_density = word_count/char_count
    
    return word_density

# Get polarity of each message
def get_polarity(msg):
    try:
        pol = TextBlob(msg).sentiment.polarity
    except:
        pol = 0.0
    return pol

Then, we create 4 Spark UDFs, each corresponding to one helper function defined in the previous cell, and define the output data type.

In [22]:
# Define Spark UDF
word_count_udf = udf(lambda msg: word_count(msg), IntegerType())
char_count_udf = udf(lambda msg: char_count(msg), IntegerType())
word_density_udf = udf(lambda msg: word_density(msg), FloatType())
get_polarity_udf = udf(lambda msg: get_polarity(msg), FloatType())

Next, we'll apply the UDFs on our dataset.

In [23]:
# Apply Spark udf into our data
df_anl = df_anl.select('date','labeled','message','language','related',
                       word_count_udf('message').alias('word_count'),
                       char_count_udf('message').alias('char_count'),
                       word_density_udf('message').alias('word_density'),
                       get_polarity_udf('message').alias('polarity'))
df_anl.show()

+-------------------+-------+--------------------+--------+-------+----------+----------+------------+------------+
|               date|labeled|             message|language|related|word_count|char_count|word_density|    polarity|
+-------------------+-------+--------------------+--------+-------+----------+----------+------------+------------+
|2010-01-01 00:00:00|      0|With the cooperat...|      en|      1|        32|       198|  0.16161616| 0.016666668|
|2010-01-01 00:00:00|      1|PEWODEN FIFTH SEC...|      ht|      1|        29|       128|   0.2265625|         0.5|
|2010-01-01 00:00:00|      1|Today on a call w...|      en|      1|        33|       161|  0.20496894|        0.15|
|2010-01-01 00:00:00|      0|YANGON, Jul 08, 2...|      en|      1|        51|       290|  0.17586207|         0.0|
|2010-01-01 00:00:00|      1|Throughout the ye...|      en|      1|        41|       247|   0.1659919| -0.29166666|
|2010-01-01 00:00:00|      0|We have no pot in...|      ht|      1|     

Earlier in the notebook, we observed that there is one or more occurrences where the message column is left blank. Since this is not useful to us, and in addition, will cause an error when we analyse the dataframe if left present (word density will be undefined as the char count, is divided by the word count, which in this case, is zero.), we will remove the case(s) where the message column is blank, with the command below.

In [24]:
# Remove unnecessary row(s)
df_anl = df_anl.where(df_anl.word_count != 0)

Now, we'll get an overview of our dataframe.

In [25]:
# Describe df
df_anl.select('word_count','char_count','word_density','polarity').describe().show()

+-------+------------------+------------------+-------------------+-------------------+
|summary|        word_count|        char_count|       word_density|           polarity|
+-------+------------------+------------------+-------------------+-------------------+
|  count|             20877|             20877|              20877|              20877|
|   mean| 23.75858600373617| 121.2066867844997|0.20574134856039755|0.04839525768272292|
| stddev|31.245594649025374|170.54404693828388|0.03954463710951681|0.22282905891324675|
|    min|                 1|                 6|        0.010416667|               -1.0|
|    max|              1686|              9144|          0.4090909|                1.0|
+-------+------------------+------------------+-------------------+-------------------+



We'll see how the features differ between different languages. We observe that messages originally in English generally has the highest word and character count, whereas messages originally in Polish have the lowest count when translated. 

Further, the average word desnity do not seem to differ by much between languages. For polarity, messages in Polish seem to have a higher average score relatively, compared to other languages.

In [26]:
# Text features by language
df_anl.groupBy('language').mean('word_count','char_count','word_density','polarity').show()

+--------+------------------+------------------+-------------------+--------------------+
|language|   avg(word_count)|   avg(char_count)|  avg(word_density)|       avg(polarity)|
+--------+------------------+------------------+-------------------+--------------------+
|      en|27.852621992866357|149.92297184488123| 0.1892106915671663|0.039803210610380765|
|      vi|               9.0|              47.0|0.19148936867713928|                 0.0|
|      nb| 9.222222222222221|              37.0| 0.2550584276517232| 0.11666666467984517|
|      sl| 10.46808510638298| 47.53191489361702|0.23426036520841273|-0.00854932389994885|
|      ro|13.714285714285714| 66.28571428571429| 0.2082196261201586|-0.02857142899717...|
|      lv|               9.0|              50.0|0.18000000715255737|                 0.0|
|      pl|               5.5|              34.0|0.15428824350237846| 0.30000001192092896|
|      sk|              11.0|40.666666666666664|0.27828853329022724|                 0.0|
|      pt|

We also observe them in related and unrelated messages. We observe that related messages are on average longer than unrelated ones, and have a slightly lower polarity score.

In [27]:
# Text features by related category
df_anl.groupBy('related').mean('word_count','char_count','word_density','polarity').show()

+-------+------------------+------------------+-------------------+-------------------+
|related|   avg(word_count)|   avg(char_count)|  avg(word_density)|      avg(polarity)|
+-------+------------------+------------------+-------------------+-------------------+
|      1|25.262631379004684|129.52633911612006| 0.2035966490507876|0.03941325727495277|
|      0|19.085185913830415|  95.3556954554397|0.21240540208288025|0.07630431029738387|
+-------+------------------+------------------+-------------------+-------------------+



Finally, we'll take a look at the messages with the highest (most positive sentiment) and lowest (most negative sentiment) polarity scores below.

In [28]:
# Messages with lowest polarity (most negative)
df_anl.orderBy('polarity').select('message').show(truncate = False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|message                                                                                                                                                                                                                      |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Ebola causes severe fever and, in the worst cases, unstoppable bleeding.                                                                                                                                                     |
|Based on what happened, if we do not learn anything from it tomorrow will be worst.                    

In [29]:
# Messages with highest polarity (most positive)
df_anl.orderBy('polarity', ascending = False).select('message').show(truncate = False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|message                                                                                                                                                                                                                                                           |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Critics point out that what Somalia will have after August 20 will not be a perfect representative democracy.                                                                                                           

We observe that messages with lower polarity scores do indeed seem more negative than those with higher polarity scores.

### End Session

This marks an end to our EDA process with Spark. Before finishing, we'll stop the Spark session with the command below.

In [30]:
# Stop Spark session
spark.stop()